<a href="https://colab.research.google.com/github/taaha3244/atom-bot/blob/main/RAGQdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf openai transformers Chroma langchain langchain_openai chromadb
!pip install --upgrade --quiet  qdrant-client

In [16]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/atomcamp.pdf")
pages = loader.load_and_split()
len(pages)
pages[0]

Document(page_content="We are a young ed-tech platform focused on | Data Science and AI | . We \ncommenced operations in 2021. Since then, we have experienced a continuous \nupward trajectory, both within the ed-tech ecosystem and within our own team. |\nT wo years down the lane, we are proud to have a team of 35 employees, with the\ndistinction of | 75% | of them being women. This remarkable achievement \nreflects the efforts of our founders, Dr. Naveed Iftikhar and Nida Mehmood, who \nhave carefully curated this team with dedication. | Our | mission | , focused on \nproviding accessible and all-inclusive education in the fields of Data Science and \nAI, has witnessed exponential growth with each successive cohort of bootcamps. \n| Through these | training initiatives | , we have nurtured a community of lifelong \nlearners who not only contribute to the Data and AI ecosystem but have also \nhoned the art of effective collaboration between diverse entities. | By offering \nimmersive | 

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(

    chunk_size=1000,
    chunk_overlap=200
)

texts = text_splitter.split_documents(pages)

print(texts[0])
print(texts[1])

page_content='We are a young ed-tech platform focused on | Data Science and AI | . We \ncommenced operations in 2021. Since then, we have experienced a continuous \nupward trajectory, both within the ed-tech ecosystem and within our own team. |\nT wo years down the lane, we are proud to have a team of 35 employees, with the\ndistinction of | 75% | of them being women. This remarkable achievement \nreflects the efforts of our founders, Dr. Naveed Iftikhar and Nida Mehmood, who \nhave carefully curated this team with dedication. | Our | mission | , focused on \nproviding accessible and all-inclusive education in the fields of Data Science and \nAI, has witnessed exponential growth with each successive cohort of bootcamps. \n| Through these | training initiatives | , we have nurtured a community of lifelong \nlearners who not only contribute to the Data and AI ecosystem but have also \nhoned the art of effective collaboration between diverse entities. | By offering' metadata={'source': '/

In [18]:
from langchain_openai import OpenAIEmbeddings
from google.colab import userdata
embeddings_model = OpenAIEmbeddings(openai_api_key=userdata.get('openai_api_key'))

In [19]:

from langchain_community.vectorstores import Qdrant


url = "https://dbd8e71d-a2a6-44db-bb8d-771198b2bdbd.us-east4-0.gcp.cloud.qdrant.io:6333"
qdrant_api_key = userdata.get('qdrant')
qdrant = Qdrant.from_documents(
    texts,
    embeddings_model,
    url=url,
    prefer_grpc=True,
    api_key=qdrant_api_key,
    collection_name="atom_bot_embeddings",
)


In [45]:
question= "What is the contact number and email address of atomcamp??"
context = qdrant.similarity_search(question)
context[0]

Document(page_content='Manager (Germany) | Education: Bachelors in Computer Sciences from FAST NU, \nIslamabad | T echnical Software Development Training from TU Munich | Close | \natomcamp is a continuous learning platform that aims to intellectually and \nprofessionally uplift Pakistan`s workforce. | atomcamp is a continuous learning \nplatform | unlocking opportunities with | Data Science. | For general inquiries: | \nteam@atomcamp.com | Home | About | Course | Publications | Home | About | \nCourse | Publications | Donâ€™t miss our future updates! | Get Subscribed \nT oday! | Please enable JavaScript in your browser to complete this form. | Email | \n* | Subscribe | For Admissions: +92-302-2278385 | For Events & Partnerships: \nfizza@atomcamp.com | 15-E Rehmat Center, Blue Area, Islamabad', metadata={'page': 4, 'source': '/content/atomcamp.pdf', '_id': 'ad0a93d2-49b0-4600-8233-8758159aaf56', '_collection_name': 'atom_bot_embeddings'})

In [46]:
from langchain.prompts import PromptTemplate
prompt=PromptTemplate(
    template=""""You ara a chatbot named 'Atom-bot' and belong to an online education platform.
    Your purpose is to spread positive image of the institute and give information to the user.Use the following pieces of context to answer the questions at the end.If
    you don't know the answer, just say don't know. do not try to make up the answer.

    <context>
    {context}
    </context>

    Question: {question}
    Helpful Answer,formatted in markdown:""",
    input_variables=["context","question"]
)


In [47]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI

llm=OpenAI(openai_api_key=userdata.get('openai_api_key'))
chain=LLMChain(llm=llm,prompt=prompt)

result=chain({
    "question":question,
    "context":"/n".join([doc.page_content for doc in context])
})

print(result['text'])


The contact number for admissions is +92-302-2278385 and the email address for general inquiries is team@atomcamp.com.
